<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/PySparkKMeanSeedVariety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pyspark

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()


data set about seeds, from UCI repository: https://archive.ics.uci.edu/ml/datasets/seeds.

Seed/wheat grain has following seven geometric parameters :

area A,

perimeter P,

compactness C = 4piA/P^2,

length of kernel,

width of kernel,

asymmetry coefficient

length of kernel groove. 

All of these parameters are real-valued continuous.


cluster these into 3 groups with K-means

In [12]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [13]:
dataset.limit(5).toPandas()

,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


**Format the Data**

In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [16]:
# Put all the columns into features column
vec_assembler = VectorAssembler(inputCols = dtaset.columns, outputCol='features')
final_data = vec_assembler.transform(dataset)

Scale the data

Scale the data to deal with the curse of dimensionality

In [17]:
from pyspark.ml.feature import StandardScaler

In [18]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

Train the model 

In [20]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures', k=3)
model = kmeans.fit(final_data)

Clustering(unsupervise learning) does not have original lables to perform any perfomance test. These is no easy way to check the performance of clustering algorithm.


At most Evaluate clustering by computing Within Set Sum of Squared Errors.

In [22]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 6.35645488 12.40730852 37.41990178 13.93860446  9.7892399   2.41585013
 12.29286107]
[ 4.078007   10.15076404 35.87686106 11.81860981  7.5430707   3.17727834
 10.39174095]
[ 4.94667359 10.96055142 37.31072041 12.42726995  8.61482659  1.79657071
 10.40733052]


In [23]:
# display cluster for each data row 
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
+----------+
only showing top 20 rows

